In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
import pystac
from pystac.extensions.datacube import Dimension, Variable
import xarray as xr

In [2]:
# Example file from https://thredds.aodn.org.au/thredds/catalog/IMOS/ANMN/NSW/CH070/gridded_timeseries/catalog.html
path = './IMOS_ANMN-NSW_TZ_20090815_CH070_FV02_TEMP-gridded-timeseries_END-20240417_C-20240608.nc'
item_id = path.split('/')[-1]  # To be set by the ingestion flow
ds = xr.open_dataset(path)
ds

<xarray.Dataset> Size: 5MB
Dimensions:     (TIME: 127263, DEPTH: 8)
Coordinates:
  * DEPTH       (DEPTH) float32 32B 0.0 10.0 20.0 30.0 40.0 50.0 60.0 70.0
  * TIME        (TIME) datetime64[ns] 1MB 2009-08-15 ... 2024-04-17T23:00:00
    LONGITUDE   float64 8B ...
    LATITUDE    float64 8B ...
Data variables:
    TEMP        (TIME, DEPTH) float32 4MB ...
    TEMP_count  (TIME) int16 255kB ...
Attributes: (12/42)
    Conventions:                   CF-1.6,IMOS-1.4
    abstract:                      Gridded Time Series Product: This file con...
    acknowledgement:               Any users of IMOS data are required to cle...
    author:                        Australian Ocean Data Network (AODN)
    author_email:                  info@aodn.org.au
    citation:                      The citation in a list of references is: "...
    ...                            ...
    source_file_download:          https://s3-ap-southeast-2.amazonaws.com/im...
    source_file_opendap:           http://thredds.aodn.org.au/thredds/dodsC/I...
    standard_name_vocabulary:      NetCDF Climate and Forecast (CF) Metadata ...
    time_coverage_end:             2024-04-17T23:00:00Z
    time_coverage_start:           2009-08-15T00:00:00Z
    title:                         Gridded Time Series Product: TEMP interpol...

The rest of this notebook deals with extracting information from the netcdf file
and converting it to a STAC item. We could explore using [rio-stac](https://github.com/developmentseed/rio-stac)
to do this, but we've had issues with that library in the past and found it
more flexible to do the extraction manually.

In [3]:
# find latitude, longitude and time coordinates
lat = None
lon = None
time = None
depth = None
for coord in ds.coords:
    if ds[coord].attrs.get('standard_name') == 'latitude':
        lat = coord
    if ds[coord].attrs.get('standard_name') == 'longitude':
        lon = coord
    if ds[coord].attrs.get('standard_name') == 'time':
        time = coord
    if ds[coord].attrs.get('standard_name') == 'depth':
        depth = coord
if lat is None or lon is None or time is None:
    raise ValueError('Could not find spatiotemporal coordinates')

# Create geoJSON box geometry
bbox = [float(ds[lon].min()), float(ds[lat].min()), float(ds[lon].max()), float(ds[lat].max())]

# TODO: work out 3D geometries

if bbox[0] == bbox[2] and bbox[1] == bbox[3]:
    geometry = dict(
        type = 'Point',
        coordinates = [bbox[0], bbox[1]]
    )
else:
    geometry = dict(
        type = 'Polygon',
        coordinates = [[
            [bbox[0], bbox[1]],
            [bbox[2], bbox[1]],
            [bbox[2], bbox[3]],
            [bbox[0], bbox[3]],
            [bbox[0], bbox[1]],
        ]]
    )

In [4]:
properties = ds.attrs

# The only required property is 'datetime' (or 'start_datetime' and 'end_datetime')
start_datetime = pd.to_datetime(ds[time].min().values, utc=True)
end_datetime = pd.to_datetime(ds[time].max().values, utc=True)
if start_datetime == end_datetime:
    single_datetime = start_datetime
    start_datetime = end_datetime = None
else:
    single_datetime = None


if 'abstract' in properties:
    ## 'description' is the preferred name in STAC
    ## For this we could use 'comment' from CF conventions or 'abstract' (from IMOS conventions?)
    properties['description'] = properties.pop('abstract')  # Should we duplicate it?
## 'title' has the same name in CF and STAC

properties['created'] = ds.attrs.pop('date_created')  # Same question here
# This should more properly be the creation date of this STAC item, while the
# creation date of the data file should be stored in the asset. If we plan to
# keep the items in sync with the data files, setting it here can make it easier
# for searching.

# Other properties we could set:
# - 'updated' (do we need it?)
# - 'deprecated' and related values: may be addressed through the version extension
#   https://github.com/stac-extensions/version
#   https://github.com/radiantearth/stac-spec/blob/master/best-practices.md#versioning-for-catalogs
# - 'provider': should always be IMOS for now, could be useful later
#   https://github.com/radiantearth/stac-spec/blob/v1.0.0/item-spec/common-metadata.md#provider-object
# - 'instrument': designed for satellite data, but could be used for other sensors
#   https://github.com/radiantearth/stac-spec/blob/v1.0.0/item-spec/common-metadata.md#instrument
# -

In [5]:
assets = dict() # Dictionary of Asset objects, keys have no predefined meaning according to STAC
assets['data'] = pystac.Asset(
    href = path,
    media_type = 'application/netcdf',
    title = 'NetCDF data',
    roles = ['data'], # other standard roles are "thumbnail", "overview", "metadata"
)

In [6]:
# UUID of the collection this item belongs to
collection = '279a50e3-21a5-4590-85a0-71f963efab82'
# I've used https://catalogue-imos.aodn.org.au/geonetwork/srv/eng/catalog.search#/metadata/279a50e3-21a5-4590-85a0-71f963efab82
# not entirely sure if it is the most appropriate for this file.
# We'll need to define a strategy to assign collections to files through geonetwork
# for the initial generation of the index, then ingestion flows should know
# which collection they are ingesting for.

In [7]:
def json_type_conversion(obj):
    # Json can have trouble with numpy types, so we need to convert them
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, xr.DataArray):
        return obj.values.tolist()
    elif isinstance(obj, np.datetime64):
        return (
            str(obj.astype("datetime64[s]")) + "Z"
        )  # numpy datetimes are assumed to always be UTC
    elif isinstance(obj, dict):
        return {k: json_type_conversion(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [json_type_conversion(v) for v in obj]
    else:
        return obj

In [8]:
item = pystac.Item(
    id=item_id,
    geometry=geometry,
    bbox=bbox,
    datetime=single_datetime,
    start_datetime=start_datetime,
    end_datetime=end_datetime,
    properties=json_type_conversion(properties),
    assets=assets,
    collection=collection,
)
item

<Item id=IMOS_ANMN-NSW_TZ_20090815_CH070_FV02_TEMP-gridded-timeseries_END-20240417_C-20240608.nc>

In [9]:
## DataCube extension for netcdf/zarr attributes and variables
item.ext.add('cube')

dimensions = dict()

for dim in ds.dims:
    dim_info = dict()
    if dim == time:
        dim_info['type'] = 'temporal'
    elif dim == lon:
        dim_info['type'] = 'spatial'
        dim_info['axis'] = 'x'
    elif dim == lat:
        dim_info['type'] = 'spatial'
        dim_info['axis'] = 'y'
    elif dim == depth:
        dim_info['type'] = 'spatial'
        dim_info['axis'] = 'z'
    else:
        dim_info['type'] = 'other'

    ## TODO: work out CRS information. Could use projection extension.
    # if dim_info['type'] == 'spatial':
    #     dim_info['reference_system'] = epsg

    dim_info['extent'] = [ds[dim].min().values, ds[dim].max().values]

    delta = ds[dim].diff(dim)
    if len(delta) > 1 and (delta[0] == delta[1:]).all():
        if dim_info['type'] == 'temporal':
            dim_info['step'] = pd.to_timedelta(delta[0].values).isoformat()
        else:
            dim_info['step'] = delta[0].values

    ## Check for monotonic dimensions, not part of the indexing but could be easy to do here
    # increasing = (ds[dim][1:] >= ds[dim][:-1]).all()
    # decreasing = (ds[dim][1:] <= ds[dim][:-1]).all()
    # assert increasing or decreasing, f"Dimension '{dim}' is not monotonic"

    description = ds[dim].attrs.get('long_name') or ds[dim].attrs.get('description')
    if description:
        dim_info['description'] = description

    unit = ds[dim].attrs.get('units') or ds[dim].attrs.get('unit')
    if unit:
        dim_info['unit'] = unit

    dimensions[dim] = Dimension(json_type_conversion(dim_info))


variables = dict()

all_vars = list(ds.data_vars) + [c for c in ds.coords if c not in ds.dims]
for var in all_vars:
    var_info = dict()
    var_info['dimensions'] = list(ds[var].dims)
    if var in ds.coords:
        var_info['type'] = 'auxiliary'
    else:
        var_info['type'] = 'data'

    description = ds[var].attrs.get('description') or ds[var].attrs.get('long_name')
    if description:
        var_info['description'] = description

    unit = ds[var].attrs.get('units') or ds[var].attrs.get('unit')
    if unit:
        var_info['unit'] = unit

    var_info['attrs'] = ds[var].attrs
    var_info['shape'] = list(ds[var].shape)

    variables[var] = Variable(json_type_conversion(var_info))

item.ext.cube.apply(dimensions, variables)

In [10]:
item

<Item id=IMOS_ANMN-NSW_TZ_20090815_CH070_FV02_TEMP-gridded-timeseries_END-20240417_C-20240608.nc>

In [11]:
item.save_object('item.json', dest_href='./item.json')
# The href will be set appropriately once we build a structured catalog